In [9]:
from jupyter_plotly_dash import JupyterDash
import import_ipynb
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### import the crud script #####
import module_script
from module_script import AnimalShelter




###########################
# Data Manipulation / Model
###########################
username = "accuser"
password = "test123"
shelter = AnimalShelter(username, password)



# was getting a typeError, to fix it _id field from ObjectId needs to be converted to a string. 
# this way it can be serialized for JSON.
df = pd.DataFrame.from_records(
        [{**record, '_id': str(record['_id'])} for record in shelter.read({})])



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#Adds in Grazioso Salvare’s logo
image_filename = 'GSLogo.png'  # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# Adds image and decodes the encoded image. 
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Center(html.Img(src = f'data:image/png;base64,{encoded_image}')),
    html.Center(html.B(html.H1('Aneudi M. SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        # Filter by all animals or animal type. 
        dcc.RadioItems(
            id = 'filter-type',
            options = [
                {'label': 'All Rescues', 'value': 'ALL'},
                {'label': 'Water Rescue', 'value': 'WATER'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'MOUNTAIN'},
                {'label': 'Disaster or Individual Tracking', 'value': 'DISASTER'}

            ],
            value = 'ALL',
            labelStyle = {'display': 'inline-block'}
        )
    ),
    html.Hr(), #Dashboard features.
    dt.DataTable(
        id = 'datatable-id',
        columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data = df.to_dict('records'),
        page_size = 10,
        selected_rows = [0],
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = "single",
        row_deletable = False,
        selected_columns = [],
        page_action = "native",
        page_current = 0
    ),

        
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className = 'row',
         style = {'display' : 'flex'},
             children = [
        html.Div(
            id = 'graph-id',
            className = 'col s12 m6',

            ),
        html.Div(
            id = 'map-id',
            className = 'col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries
        
    if filter_type == 'ALL':
        query = {}
    elif filter_type == 'WATER':
        query = {'rescue_type': 'Water Rescue'}
    elif filter_type == 'MOUNTAIN':
        query = {'rescue_type': 'Mountain or Wilderness Rescue'}
    elif filter_type == 'DISASTER':
        query = {'rescue_type': 'Disaster or Individual Tracking'}
        
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data = df.to_dict('records')
        
        
    return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#Piechart to show the percentage of animal type in the data. 
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    df = pd.DataFrame(viewData)
    if not df.empty:
        pie_chart = px.pie(df, names = 'animal_type', title = 'Animal Type Distribution')
        return [dcc.Graph(figure = pie_chart)]
    return []

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'selected_rows')])
def update_map(viewData, selected_rows):
#FIXME Add in the code for your geolocation chart
    if viewData is None or len(viewData) == 0:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    
    if selected_rows is None or len(viewData) == 0:
        row = 0
    
    else:
        row = selected_rows[0]
        
    lat, lon = 30.75, -97.486
    
    if not pd.isna(dff.iloc[row, 13]) and not pd.isna(dff.iloc[row, 14]):
        lat = dff.iloc[row, 13]
        lon = dff.iloc[row, 14]

    # Austin TX is at [30.75, -97.48]

    return[
        dl.Map(style={'width': '1000px', 'height': '500px'}, center = [lat, lon], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position = [lat, lon], children = [
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

app